- 다음 영화 정보를 수집한다.

In [1]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os
from urllib.request import urlopen
from IPython.display import clear_output

In [2]:
# 요청 함수.
def getSource(site) :
    # 헤더 정보.
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
    }
    
    # 요청.
    response = requests.get(site, headers = header_info)
    # print(response.text)
    
    # bs4 객체 생성.
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    return soup

In [3]:
# 한 페이지의 데이터를 수집해 저장하는 함수.
def getData(soup) :
    
    # 데이터를 담을 딕셔너리 생성.
    data_dict = {
        '제목'     : [],
        '평점'     : [],
        '개봉날짜' : [],
        '예매율'   : []
                 }
    
    # 전체를 가지고 있는 태그를 가져오기.
    a1 = soup.select_one('#mArticle > ul.list_movie')
    # print(a1)
    
    # 영화들을 가져오기.
    a2 = a1.select('li')
    # print(a2)
    
    # 영화의 수 만큼 반복.
    for a3 in a2 :
        # 영화 제목을 가져오기.
        a4 = a3.select_one('div.wrap_movie > div > a')
        data1 = a4.text.strip()
        # print(data1)
        
        # 평점을 가져오기.
        a5 = a3.select('div.wrap_movie > span.info_grade > a > span.wrap_grade.grade_netizen > span.num_grade')
        b1 = int(a5[0].text)
        b2 = int(a5[2].text)
        data2 = f'{b1}.{b2}'
        # print(data2)
        
        # 개봉날짜와 예매율.
        a6 = a3.select_one('div.wrap_movie > span.info_state')
        b3 = a6.text.split('・')
        
        data3 = b3[0].strip()
        
        if len(b3) == 2 :    
            data4 = b3[1].strip()
        else :
            data4 = np.nan
        
        # 개봉날짜에서 '개봉일' 문자열 제거.
        data3 = data3.replace('개봉', '').strip()
        data3 = data3.replace('재', '').strip()
        
        # 예매율.
        if len(b3) == 2 :
            data4 = data4.replace('예매율', '').strip()
            data4 = data4.replace('%', '').strip()
        
        #print(data1)
        #print(data2)
        #print(data3)
        #print(data4)
        
        # 영화 포스터.
        a7 = a3.select_one('div.info_movie > span > img')
        
        # src 속성값을 가져오기.
        src_attr = a7.attrs['src']
        # print(src_attr)
        
        if len(src_attr) > 0 :
            # 포스터 이미지가 존재할 경우.
            # 이미지 데이터를 읽기.
            with urlopen('https:' + src_attr) as f1 :
                img_data = f1.read()
                
            # 영화 제목의 특수문자를 제거.
            filename = data1.replace(':', '-')
            filename = filename.replace('/', ' ')
            filename = filename.replace(';', '')
        
            # 저장.
            with open(f'poster/{filename}.jpg', 'wb') as f2 :
                f2.write(img_data)
        
        data_dict['제목'].append(data1)
        data_dict['평점'].append(data2)
        data_dict['개봉날짜'].append(data3)
        data_dict['예매율'].append(data4)
        
    # 데이터 프레임 생성.
    df1 = pd.DataFrame(data_dict)
    # display(df1)
    
    if os.path.exists('movie.csv') == False :
        # 파일이 없을 경우.
        df1.to_csv('movie.csv', encoding='utf-8-sig', index=False)
    else :
        df1.to_csv('movie.csv', encoding='utf-8-sig', index=False, header=False, mode='a')

In [4]:
# 다음 페이지 존재 여부를 확인하는 함수.
def getNext(soup) :
    # 페이지 번호 중 a 태그를 가져오기.
    a_tag = soup.select('#mArticle > div.paging_movie > span > a')
    # print(a_tag)
    
    # 페이지 번호 중 em 태그를 가져오기.
    em_tag = soup.select_one('#mArticle > div.paging_movie > span > em')
    # print(em_tag)
    
    # a 태그 중 제일 마지막 태그의 숫자 값을 가져오기.
    a1 = a_tag[-1].text.strip()
    
    # em 태그의 숫자값을 가져오기.
    a2 = em_tag.text.strip()
    a2 = a2.replace('현재페이지', '')
    # print(a1)
    # print(a2)
    
    if int(a1) > int(a2) :
        # 마지막 a 태그의 숫자가 em 태그의 숫자보다 더 큰 경우.
        return True
    else :
        return False

In [5]:
page_name = 'released'
page = 1

while True :
    time.sleep(1)
    
    clear_output(wait=True)
    
    site = f'https://movie.daum.net/premovie/{page_name}?reservationOnly=N&sort=reservation&page={page}'

    print(f'{page_name} : {page} 수집중')
    
    soup = getSource(site)
    getData(soup)
    chk = getNext(soup)
    
    if chk == False :
        if page_name == 'released' :
            page_name = 'scheduled'
            page = 1
        else :
            print('수집완료')
            break
    else :
        page += 1

scheduled : 6 수집중
수집완료


In [6]:
# 데이터 프레임을 생성.
df = pd.read_csv('movie.csv')
df

,제목,평점,개봉날짜,예매율
0,극장판 귀멸의 칼날: 무한열차편,5.5,21.01.27,31.6
1,소울,8.7,21.01.20,25.5
2,어니스트 씨프,6.8,21.02.03,5.9
3,해피 투게더,8.3,21.02.04,2.9
4,페어웰,8.3,21.02.04,1.8
...,...,...,...,...
622,꿈꾸는 고양이,10.0,21.06.04,NaN
623,달이 지는 밤,10.0,21.09.01,NaN
624,물방울을 그리는 남자,0.0,21.10.19,NaN
625,007 노 타임 투 다이,7.6,21.10,NaN
